# Collatz Proof

In this version, we tweak the function so that for odd we also divide by 2 once.

In this version we are discovering the generator function for "the next bits"

## Predictive Power

In [1]:
import sys, io
import math
import numpy as np
import pandas as pd
from scipy.optimize import nnls

pd.set_option('display.max_rows', None)

In [2]:
def collatzNext(n):
    if n %2 == 0:
        return n//2
    return (3*n + 1)//2

def collatzIter(start, iterations):
    n = start
    count = 0
    while count < iterations:
        if n %2 == 0:
            m = n//2
            yield n
            n = m
        else:
            m = (3*n + 1)//2
            yield n
            n = m
        count = count + 1
    #
    return 1


In [3]:
def blocks(sequence):
    for n in sequence:
        if  n %2 == 0:
            yield '□'
        else:
            yield '■'
    

In [4]:
def split(q, L_in):
    L_out = []
    for elem in L_in:
        L_out.append(elem)
        L_out.append(elem + 2**q)
    q = q + 1
    return q, L_out
#
def gen_partitions():
    L = [1]
    for p in range(1,8,1):
        q, L = split(p, L)
        print("%d   %s"%(q, L))
#
gen_partitions()

2   [1, 3]
3   [1, 5, 3, 7]
4   [1, 9, 5, 13, 3, 11, 7, 15]
5   [1, 17, 9, 25, 5, 21, 13, 29, 3, 19, 11, 27, 7, 23, 15, 31]
6   [1, 33, 17, 49, 9, 41, 25, 57, 5, 37, 21, 53, 13, 45, 29, 61, 3, 35, 19, 51, 11, 43, 27, 59, 7, 39, 23, 55, 15, 47, 31, 63]
7   [1, 65, 33, 97, 17, 81, 49, 113, 9, 73, 41, 105, 25, 89, 57, 121, 5, 69, 37, 101, 21, 85, 53, 117, 13, 77, 45, 109, 29, 93, 61, 125, 3, 67, 35, 99, 19, 83, 51, 115, 11, 75, 43, 107, 27, 91, 59, 123, 7, 71, 39, 103, 23, 87, 55, 119, 15, 79, 47, 111, 31, 95, 63, 127]
8   [1, 129, 65, 193, 33, 161, 97, 225, 17, 145, 81, 209, 49, 177, 113, 241, 9, 137, 73, 201, 41, 169, 105, 233, 25, 153, 89, 217, 57, 185, 121, 249, 5, 133, 69, 197, 37, 165, 101, 229, 21, 149, 85, 213, 53, 181, 117, 245, 13, 141, 77, 205, 45, 173, 109, 237, 29, 157, 93, 221, 61, 189, 125, 253, 3, 131, 67, 195, 35, 163, 99, 227, 19, 147, 83, 211, 51, 179, 115, 243, 11, 139, 75, 203, 43, 171, 107, 235, 27, 155, 91, 219, 59, 187, 123, 251, 7, 135, 71, 199, 39, 167, 103, 231,

In [5]:
def next_bits(p, L):
    B = []
    for elem in L:
        x = 2**p + elem
        for i in range(p-1):
            x = collatzNext(x)
        if x % 2 == 0:
            bit = 0
        else:
            bit = 1
        B.append(bit)
        
        if elem == 5:
            print("next bit for 5 is %d"%(bit))
    return B

def numFromBits(A):
    B = list(reversed(A))
    s = "".join(map(str, B))
    n = int(s, base=2)
    return n
#

def gen_partition_next_bit():
    L = [1]
    for p in range(1,8,1):
        q, L = split(p, L)
        print("%d   %s"%(q, L))
        B = next_bits(q, L)
        print("%d   %s"%(q, B))
        n = numFromBits(B)
        print("%d   %d"%(q,n))
#
gen_partition_next_bit()

2   [1, 3]
2   [0, 1]
2   2
3   [1, 5, 3, 7]
next bit for 5 is 0
3   [1, 0, 0, 1]
3   9
4   [1, 9, 5, 13, 3, 11, 7, 15]
next bit for 5 is 0
4   [0, 1, 0, 1, 0, 1, 0, 1]
4   170
5   [1, 17, 9, 25, 5, 21, 13, 29, 3, 19, 11, 27, 7, 23, 15, 31]
next bit for 5 is 1
5   [1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1]
5   39573
6   [1, 33, 17, 49, 9, 41, 25, 57, 5, 37, 21, 53, 13, 45, 29, 61, 3, 35, 19, 51, 11, 43, 27, 59, 7, 39, 23, 55, 15, 47, 31, 63]
next bit for 5 is 0
6   [0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1]
6   2858789542
7   [1, 65, 33, 97, 17, 81, 49, 113, 9, 73, 41, 105, 25, 89, 57, 121, 5, 69, 37, 101, 21, 85, 53, 117, 13, 77, 45, 109, 29, 93, 61, 125, 3, 67, 35, 99, 19, 83, 51, 115, 11, 75, 43, 107, 27, 91, 59, 123, 7, 71, 39, 103, 23, 87, 55, 119, 15, 79, 47, 111, 31, 95, 63, 127]
next bit for 5 is 1
7   [1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 

In [7]:
for i in range(0,64,1):
    o = 2*i+1
    e = 3*o + 1
    n = 0
    while e % 2 == 0:
        e = e/2
        n = n+1
    print("%4d %4d"%(o, e))

   1    1
   3    5
   5    1
   7   11
   9    7
  11   17
  13    5
  15   23
  17   13
  19   29
  21    1
  23   35
  25   19
  27   41
  29   11
  31   47
  33   25
  35   53
  37    7
  39   59
  41   31
  43   65
  45   17
  47   71
  49   37
  51   77
  53    5
  55   83
  57   43
  59   89
  61   23
  63   95
  65   49
  67  101
  69   13
  71  107
  73   55
  75  113
  77   29
  79  119
  81   61
  83  125
  85    1
  87  131
  89   67
  91  137
  93   35
  95  143
  97   73
  99  149
 101   19
 103  155
 105   79
 107  161
 109   41
 111  167
 113   85
 115  173
 117   11
 119  179
 121   91
 123  185
 125   47
 127  191


In [9]:
o = 0
p = 0
for i in range(0,16,1):
    o = 4**p + o
    p = p+1
    e = 3*o + 1
    n = 0
    while e % 2 == 0:
        e = e/2
        n = n+1
    print("%4d %4d"%(o, e))

   1    1
   5    1
  21    1
  85    1
 341    1
1365    1
5461    1
21845    1
87381    1
349525    1
1398101    1
5592405    1
22369621    1
89478485    1
357913941    1
1431655765    1


In [10]:
print(list(collatzIter(64+29, 40)))

[93, 140, 70, 35, 53, 80, 40, 20, 10, 5, 8, 4, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1]


In [11]:
print(list(collatzIter(128+29, 40)))

[157, 236, 118, 59, 89, 134, 67, 101, 152, 76, 38, 19, 29, 44, 22, 11, 17, 26, 13, 20, 10, 5, 8, 4, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1]


In [22]:
def getResidue(num):
    p = math.log(num, 2)
    p = int(math.floor(p))
    return num - 2**p

# What are the R3 translations?
for i in range(2,64,1):
    o = 2**i + 3
    c = o
    while c == o or c % 2 == 0:
        c = collatzNext(c)
        r = getResidue(c)
    print("%d\t%d\t%d\t%d\t%f\t%d"%(i, o, c, r, o/r, o - 2*r))


2	7	11	3	2.333333	1
3	11	17	1	11.000000	9
4	19	29	13	1.461538	-7
5	35	53	21	1.666667	-7
6	67	101	37	1.810811	-7
7	131	197	69	1.898551	-7
8	259	389	133	1.947368	-7
9	515	773	261	1.973180	-7
10	1027	1541	517	1.986460	-7
11	2051	3077	1029	1.993197	-7
12	4099	6149	2053	1.996590	-7
13	8195	12293	4101	1.998293	-7
14	16387	24581	8197	1.999146	-7
15	32771	49157	16389	1.999573	-7
16	65539	98309	32773	1.999786	-7
17	131075	196613	65541	1.999893	-7
18	262147	393221	131077	1.999947	-7
19	524291	786437	262149	1.999973	-7
20	1048579	1572869	524293	1.999987	-7
21	2097155	3145733	1048581	1.999993	-7
22	4194307	6291461	2097157	1.999997	-7
23	8388611	12582917	4194309	1.999998	-7
24	16777219	25165829	8388613	1.999999	-7
25	33554435	50331653	16777221	2.000000	-7
26	67108867	100663301	33554437	2.000000	-7
27	134217731	201326597	67108869	2.000000	-7
28	268435459	402653189	134217733	2.000000	-7
29	536870915	805306373	268435461	2.000000	-7
30	1073741827	1610612741	536870917	2.000000	-7
31	2147483651	322122547